In [1]:
import pickle
import pandas as pd
import pyvis
import os

In [3]:
root_path = r"C:\Users\Joycelyn\Documents\Synapxe\07 GenAI for healthhub\Github backup\cluster experiments\weighted emb vs sim (658)"
emb_path_title = os.path.join(root_path,"nomic_weighted_emb (0.7 body, 0.3 title)")
emb_path_metadesc = os.path.join(root_path,"nomic_weighted_emb (0.7 body, 0.3 meta desc)")

with open(os.path.join(emb_path_title,"neo4j_predicted_clusters.pkl"), "rb") as f:
    title_results = pickle.load(f)

with open(os.path.join(emb_path_metadesc,"neo4j_predicted_clusters.pkl"), "rb") as f:
    desc_results = pickle.load(f)

## Extract clusters <= 5 articles

In [16]:
title_results = pd.DataFrame(title_results)
title_results['count'] = title_results['titles'].apply(len)
title_results_filtered = title_results[title_results["count"] < 6]
print(title_results_filtered.shape)

(59, 4)


In [17]:
desc_results = pd.DataFrame(desc_results)
desc_results['count'] = desc_results['titles'].apply(len)
desc_results_filtered = desc_results[desc_results["count"] < 6]
print(desc_results_filtered.shape)

(57, 4)


In [19]:
save_path = os.path.join(root_path, "body_with_title_vs_metadesc_size5.xlsx")
with pd.ExcelWriter(save_path) as writer:
    title_results_filtered.to_excel(writer, sheet_name='title_results_filtered', index=False)
    desc_results_filtered.to_excel(writer, sheet_name='desc_results_filtered', index=False)

## Cluster size, bin 5

In [102]:
def get_cluster_size(pred_cluster):
    grouped_counts = pred_cluster.groupby('cluster').size()
    filtered_grouped_counts = grouped_counts[grouped_counts != 1]
    bins = range(1, filtered_grouped_counts.max() + 5, 5)
    labels = [f"{i}-{i+4}" for i in bins[:-1]]
    labels[0] = '2-5'
    binned_counts = pd.cut(filtered_grouped_counts, bins=bins, labels=labels, right=False)
    banded_counts = binned_counts.value_counts().sort_index()
    cluster_size_df2 = pd.DataFrame(banded_counts).reset_index().rename(columns={'index':"Cluster size",'count':"Num of clusters"})
    cluster_size_df2 = cluster_size_df2.sort_index().reset_index(drop=True)
    return cluster_size_df2

In [103]:
df = pd.read_csv(r"C:\Users\Joycelyn\Documents\Synapxe\07 GenAI for healthhub\Github\healthhub-content-optimization\content-optimization\data\07_model_output\predicted_cluster.csv")
get_cluster_size(df)

,Cluster size,Num of clusters
0,2-5,58
1,6-10,3
